In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns


# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

import env
import k_wrangle

In [2]:
df = k_wrangle.get_curriculum_logs()
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [3]:
df = k_wrangle.prep_curriculum_data()
df.head()

,time,endpoint,user_id,cohort_id,ip_address,id,cohort_name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name,program
date,,,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0,Java Full Stack Web Development,Web Development


# Explore

### Question 1: Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [4]:
# curriculum access value counts per program

df.program.value_counts()


Web Development    743918
Data Science       103412
Name: program, dtype: int64

In [5]:
# top 5 overall pages
print('10 Most Popular Pages in Web Dev Program & DS')

df.groupby(['program'])['endpoint'].value_counts().head(10)

10 Most Popular Pages in Web Dev Program & DS


program       endpoint                                
Data Science  /                                           8358
              search/search_index.json                    2203
              classification/overview                     1785
              1-fundamentals/modern-data-scientist.jpg    1655
              1-fundamentals/AI-ML-DL-timeline.jpg        1651
              1-fundamentals/1.1-intro-to-data-science    1633
              classification/scale_features_or_not.svg    1590
              fundamentals/AI-ML-DL-timeline.jpg          1443
              fundamentals/modern-data-scientist.jpg      1438
              sql/mysql-overview                          1424
Name: endpoint, dtype: int64

In [6]:
page_views = df.groupby(['cohort_name'])['endpoint'].agg(['count', 'unique'])
page_views

,count,unique
cohort_name,,
Andromeda,25359,"[assets/js/pdfmake.min.js.map, /, toc, html-cs..."
Apex,33568,"[/, toc, html-css, html-css/introduction, java..."
Apollo,5,"[/, content/html-css, content/html-css/gitbook..."
Arches,8890,"[/, javascript-i, javascript-i/functions, appe..."
Badlands,93,"[/, prework/fundamentals, prework/fundamentals..."
Bash,17713,"[/, html-css, toc, html-css/introduction, sear..."
Bayes,26538,"[/, 3-sql/1-mysql-overview, 2-storytelling/bad..."
Betelgeuse,29356,"[/, html-css, toc, spring, appendix, appendix/..."
Ceres,40730,"[/, toc, quiz, javascript-i, spring, java-i, j..."


In [20]:
# create ds dataframe
DS_df = df[(df.program == 'Data Science')]
DS_df.head(5)

,time,endpoint,user_id,cohort_id,ip_address,id,cohort_name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name,program
date,,,,,,,,,,,,,,,,
2019-08-20,09:39:58,/,466,34.0,97.105.19.58,34.0,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,None,3.0,Data Science,Data Science
2019-08-20,09:39:59,/,467,34.0,97.105.19.58,34.0,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,None,3.0,Data Science,Data Science
2019-08-20,09:39:59,/,468,34.0,97.105.19.58,34.0,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,None,3.0,Data Science,Data Science
2019-08-20,09:40:02,/,469,34.0,97.105.19.58,34.0,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,None,3.0,Data Science,Data Science
2019-08-20,09:40:08,/,470,34.0,97.105.19.58,34.0,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,None,3.0,Data Science,Data Science


In [24]:
# top 5 overall pages
print('5 Most Popular Pages in Data Science Program')

DS_df.endpoint.value_counts().head(5)

5 Most Popular Pages in Data Science Program


/                                           8358
search/search_index.json                    2203
classification/overview                     1785
1-fundamentals/modern-data-scientist.jpg    1655
1-fundamentals/AI-ML-DL-timeline.jpg        1651
Name: endpoint, dtype: int64

In [26]:
# create Web dev df
Wb_df = df[(df.program == 'Web Development')]
Wb_df.head(5)

,time,endpoint,user_id,cohort_id,ip_address,id,cohort_name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name,program
date,,,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0,Java Full Stack Web Development,Web Development


In [27]:
# top 5 overall pages
print('5 Most Popular Pages in Web Development Program')

Wb_df.endpoint.value_counts().head(5)

5 Most Popular Pages in Web Development Program


/                           37496
javascript-i                18193
toc                         17580
search/search_index.json    15331
java-iii                    13162
Name: endpoint, dtype: int64

In [7]:
subset = df[(df.endpoint != '/') & (df.cohort_name != 'Staff') & (df.program != 'none')]


In [8]:
subset[['base', 'endpoint']] = subset.endpoint.str.split(pat='/', n=1, expand=True)


In [9]:
subset.base = subset.base.str.lstrip('0123456789.- ')
subset.head()

,time,endpoint,user_id,cohort_id,ip_address,id,cohort_name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name,program,base
date,,,,,,,,,,,,,,,,,
2018-01-26,09:56:02,None,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development,java-ii
2018-01-26,09:56:05,object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development,java-ii
2018-01-26,09:56:06,object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0,PHP Full Stack Web Development,Web Development,slides
2018-01-26,09:56:24,conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0,Java Full Stack Web Development,Web Development,javascript-i
2018-01-26,09:56:41,loops,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0,Java Full Stack Web Development,Web Development,javascript-i


In [10]:
ds_curr = subset[subset.program =='Web Development'].base.value_counts().sort_values(ascending=False).to_frame()
ds_curr.head(10)

,base
javascript-i,107525
html-css,76142
mysql,73732
jquery,54080
spring,49907
java-iii,49810
java-ii,48698
java-i,37067
javascript-ii,34823
appendix,26787


In [11]:
ds_curr = subset[subset.program =='Data Science'].base.value_counts().sort_values(ascending=False).to_frame()
ds_curr.head(10)

,base
fundamentals,16691
sql,13670
classification,11840
python,10455
regression,7507
stats,6660
anomaly-detection,4332
clustering,4272
appendix,3944
timeseries,3315


### 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [12]:
# least referred to lessons across all cohorts for Web Development program

ds_curr = subset[subset.program =='Web Development'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

,base
ajax-api-request.html,1
team,1
javascri,1
_file_paths,1
capsones,1
javascript-functions,1
_Acquirecsv,1
_ParseText,1
_RidgeRegression,1
_Intro_Stats,1


In [13]:
# least referred to lessons across all cohorts for Data Science program

ds_curr = subset[subset.program =='Data Science'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

,base
mkdocs,1
introduction-to-python,1
viz.py,1
spark-local-mode.svg,1
imports,1
diagram-of-ds-pipeline-fraud-example.jpeg,1
misleading1_fox.jpg,1
sql.mysql-overview,1
Index.html,1
case-statements,1


In [14]:
# cohort Andromeda
ds_curr = subset[subset.cohort_name =='Andromeda'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

,base
documentation,1
sessions-and-cookies,1
tools,1
sql,1
quiz,1
html,1
jsp-and-jstl,1
testing-user-functions,1
capstone,1
events,1


In [15]:
# Cohort Apex
ds_curr = subset[subset.cohort_name =='Apex'].base.value_counts().sort_values(ascending=True).to_frame()
ds_curr.head(60)

,base
capstone-workbook,1
forms,1
interfaces-and-abstract-classes,1
tools,2
user-input,2
css-i,3
setup,3
passwords,3
finish-the-adlister,8
elements,23


### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [16]:
df.user_id.nunique()


981

In [17]:
df.user_id.value_counts()

11     17913
64     16347
53     12329
314     7783
1       7404
       ...  
652        1
918        1
649        1
952        1
212        1
Name: user_id, Length: 981, dtype: int64

In [18]:
page_views = df.groupby(['user_id'])['endpoint'].agg(['count', 'unique'])
page_views

,count,unique
user_id,,
1,7404,"[/, java-ii, java-ii/object-oriented-programmi..."
2,1541,"[javascript-i/conditionals, javascript-i/loops..."
3,1562,"[javascript-i/conditionals, javascript-i/funct..."
4,692,"[javascript-i/functions, javascript-i/introduc..."
5,1701,"[mkdocs/search_index.json, appendix/further-re..."
...,...,...
977,86,"[/, javascript-i, javascript-i/javascript-with..."
978,55,"[/, toc, javascript-i, javascript-i/introducti..."
979,127,"[/, toc, javascript-i/introduction, javascript..."
